In [53]:
import pandas as pd
from surprise import SVD, Reader, SVDpp
from surprise import Dataset
from surprise.model_selection import cross_validate

In [26]:
ingr_map = pd.read_pickle("../datasets/our_ingr_map.pkl")
recipes = pd.read_pickle("../datasets/our_recipes.pkl")
interactions = pd.read_pickle("../datasets/our_interactions.pkl")

In [27]:
interactions.head(5)

,user_id,recipe_id,date,rating,u
0,2046,517,2000-02-25,5.0,22095
1,868626,517,2009-07-24,5.0,11611
2,1773,7435,2000-03-13,5.0,24732
3,16346,7435,2001-08-23,0.0,15023
4,10649,7435,2001-12-06,3.0,17766


In [29]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(interactions[['user_id', 'recipe_id', 'rating']], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
# cross_validate(SVD(), data, cv=2)

In [30]:
trainset = data.build_full_trainset()

In [31]:
# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
%time algo.fit(trainset)

CPU times: user 33.6 s, sys: 209 ms, total: 33.8 s
Wall time: 34.5 s


In [36]:
def get_recommendations_for_user(algo, dataset, user_id):
    ratings = []
    for recipe_id in dataset["recipe_id"].unique():
        pred = algo.predict(user_id, recipe_id)
        ratings += [(recipe_id, pred)]
    return sorted(ratings, key=lambda x: x[1], reverse=True)

In [47]:
list(filter(lambda x: x[0] ==115849, get_recommendations_for_user(algo, interactions, 603504)))

[(115849,
  Prediction(uid=603504, iid=115849, r_ui=None, est=2.789493308338787, details={'was_impossible': False}))]

In [57]:
cross_validate(SVD(), data, cv=5)

{'test_rmse': array([0.93408599, 0.94920781, 0.93079432, 0.93617153, 0.93450808]),
 'test_mae': array([0.55707155, 0.56372052, 0.55657858, 0.5582774 , 0.5575665 ]),
 'fit_time': (20.539171934127808,
  21.628021001815796,
  20.57081699371338,
  20.00734806060791,
  19.868424892425537),
 'test_time': (0.6464390754699707,
  1.827733039855957,
  0.5955562591552734,
  1.5630760192871094,
  0.6073510646820068)}